# 카카오 로컬 API: 도로명 주소 --> 위도, 경도

In [2]:
import requests
from urllib.parse import quote
import pandas as pd

with open('kakaoapikey.txt') as f:
    api_key = f.read()
len(api_key)

32

In [3]:
addr ='서울특별시 중구 세종대로 110(태평로1가)'
search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
url = f'{search_url}?query={quote(addr)}' # & 붙인다
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%A4%91%EA%B5%AC%20%EC%84%B8%EC%A2%85%EB%8C%80%EB%A1%9C%20110%28%ED%83%9C%ED%8F%89%EB%A1%9C1%EA%B0%80%29'

In [4]:
#"Authorization: KakaoAK {REST_API_KEY}"
result = requests.get(url,
            headers={"Authorization": f'KakaoAK {api_key}'}).json()
result

{'documents': [{'address': {'address_name': '서울 중구 태평로1가 31',
    'b_code': '1114010300',
    'h_code': '1114055000',
    'main_address_no': '31',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '중구',
    'region_3depth_h_name': '명동',
    'region_3depth_name': '태평로1가',
    'sub_address_no': '',
    'x': '126.977829174031',
    'y': '37.5663174209601'},
   'address_name': '서울 중구 세종대로 110',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 중구 세종대로 110',
    'building_name': '서울특별시청',
    'main_building_no': '110',
    'region_1depth_name': '서울',
    'region_2depth_name': '중구',
    'region_3depth_name': '태평로1가',
    'road_name': '세종대로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.977829174031',
    'y': '37.5663174209601',
    'zone_no': '04524'},
   'x': '126.977829174031',
   'y': '37.5663174209601'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [5]:
lng ,lat = float(result['documents'][0]['x']), float(result['documents'][0]['y'])
lng, lat

(126.977829174031, 37.5663174209601)

## 공공기관.csv 파일에 위도 경도 추가하기

In [6]:
df = pd.read_csv('공공기관.csv')
df

,공공기관,도로명 주소
0,서울특별시청,서울특별시 중구 세종대로 110(태평로1가)
1,노원구청,서울특별시 노원구 노해로 437(상계동)
2,송파구청,서울특별시 송파구 올림픽로 326(신천동)
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동)
4,강서구청,서울특별시 강서구 남부순환로 208(외발산동)


In [7]:
lat_list = []
lng_list = []
for juso in df['도로명 주소']:
    url = f'{search_url}?query={quote(juso)}' # & 붙인다
    result = requests.get(url,
            headers={"Authorization": f'KakaoAK {api_key}'}).json()
    lng = float(result['documents'][0]['x'])
    lat = float(result['documents'][0]['y'])
    lat_list.append(lat)
    lng_list.append(lng)

lat_list, lng_list

([37.5663174209601,
  37.6545190418039,
  37.514477182474,
  37.5634263131969,
  37.5468670862325],
 [126.977829174031,
  127.056297212439,
  127.105859984389,
  126.903356973848,
  126.820414204647])

In [8]:
df['위도'] = lat_list
df['경도'] = lng_list

df.to_csv('공공기관2.csv')

## 지도위에 공공기관 표시하기

In [9]:
import folium
import pandas as pd
folium.__version__

'unknown'

In [10]:
map = folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=10)

for i in df.index:
        folium.Circle(
                radius=200,
                location = [df.위도[i], df.경도[i]],
                popup = folium.Popup(f'{ df["도로명 주소"][i] }', max_width=200),
                tooltip = df.공공기관[i],
                color= '#3186cc',
                fill=True,
                fillcolor = '#3186cc'
        ).add_to(map)
title = '<h3 align="center" style="font-size:20px">서울시 구청 안내</h3>'
map.get_root().html.add_child(folium.Element(title))        

map

In [13]:
df2 = pd.read_csv('data/서울공원.csv', encoding='euc-kr')

df2.head()

,연번,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107,450395.554,126.990377,37.550140,http://parks.seoul.go.kr/template/sub/namsan.do
1,2,길동생태공원,길동생태공원은 생물의 서식처를 제공하고 종다양성을 증진시키며 자연생태계의 생물들을 ...,80683㎡,1999.5.20,탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터,소나무 보리수 등 64종 31800주 산국 부들 등 138종 192800본,http://parks.seoul.go.kr/template/common/img/p...,NaN,매주 월요일은 휴관입니다 사전예약 후 입장하실 수 있습니다. ▶ 인터넷 예약 바로가...,http://parks.seoul.go.kr/file/info/view.do?fId...,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),동부공원녹지사업소 길동생태공원,02-489-2770,213554.120,448852.675,127.154779,37.540394,http://parks.seoul.go.kr/template/sub/gildong.do
2,3,서울대공원,서울대공원은 세계 각국의 야생동물들이 살아 숨 쉬는 서울동물원과 다양한 재미와 즐거...,9132690m²,1984.5.1,동물원 식물원 테마가든(장미원 어린이동물원 피크닉장) 치유숲 산림욕장 캠핑장 국립현...,NaN,NaN,지하철 4호선 대공원역 하차(2번 출구) 도보 15분 정도 문의처 02)500-73...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,과천시,경기도 과천시 대공원광장로 102,전략기획실,02-500-7032,200994.267,437163.981,127.019846,37.426449,http://grandpark.seoul.go.kr/
3,4,서울숲,당초 골프장 승마장 등이 있던 뚝섬일대를 주거업무 지역으로 개발할 경우 약 4조원에...,480994㎡,2005.6.18,4개 테마공원: 문화예술공원 자연생태숲 자연체험학습원 습지생태원 주요시설: 가족마당...,수 목 : 소나무 섬잣나무 계수나무 외 95종 415795주 식물원 : 선인장 등 ...,http://parks.seoul.go.kr/template/common/img/p...,NaN,지하철 분당선 서울숲역 3번출구 도보로 약5분 2호선 뚝섬역 8번출구 도보로 약 1...,http://parks.seoul.go.kr/file/info/view.do?fId...,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),서울숲 관리사무소,02-460-2905,203695.432,449290.726,127.041798,37.543072,http://parks.seoul.go.kr/template/sub/seoulfor...
4,5,월드컵공원,월드컵공원은 서울의 서쪽에 위치하여 1978년부터 1993년까지 15년간 서울시민이...,2284085㎡,2002.5.1,평화의 공원 월드컵공원전시관(879㎡) 유니세프광장(2400㎡) 평화광장(5217㎡...,NaN,http://parks.seoul.go.kr/template/common/img/p...,지하철 6호선 월드컵경기장역 하차 → 1번 출구로 나온 후 직진 → 큰길(도로)이 ...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),월드컵공원,02-300-5500,190658.070,451598.831,126.878907,37.571805,http://parks.seoul.go.kr/template/sub/worldcup...
